In [ ]:
#this script is used for analyzing things we get as the result of dnds cv

In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import numpy

from collections import Counter

pathPrefix = '/Users/friedman/Desktop/mnt'

sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import clonality_analysis_util
import re


In [2]:
allImpactMuts = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,36,39,85,136) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
endometrialHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'Hypermutated')
colorectalHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'Hypermutated')
gliomaHyperIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'Hypermutated')

endometrialNormalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'Normal')
colorectalNormalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'Normal')
gliomaNormalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'Normal')

In [5]:
endometrialHyperMuts = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(endometrialHyperIds)]
colorectalHyperMuts = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(colorectalHyperIds)]
gliomaHyperMuts = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(gliomaHyperIds)]

endometrialNormalMuts = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(endometrialNormalIds)]
colorectalNormalMuts = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(colorectalNormalIds)]
gliomaNormalMuts = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(gliomaNormalIds)]

In [12]:

#dndscvNonHyper = pd.read_table('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/allNonHypermutantImpactDndsCVresults.tsv')
qValDict = dict(zip(dndscvNonHyper['gene_name'], dndscvNonHyper['qglobal_cv']))

In [9]:
endometrialHotspotMuts = endometrialHyperMuts[endometrialHyperMuts['is-a-hotspot'] == 'Y']

In [ ]:
#TODO--exclude genes that are siginificant in Endometrial normal 

In [33]:
oncogenicMuts = colorectalHyperMuts[colorectalHyperMuts['oncogenic'].notnull()]

#oncogenicMuts = endometrialHyperMuts[endometrialHyperMuts['oncogenic'].notnull()]
listOfDicts = []
for gene in set(oncogenicMuts['Hugo_Symbol']):
    geneMaf = oncogenicMuts[oncogenicMuts['Hugo_Symbol'] == gene]
    nMutOnc = geneMaf.shape[0]
    nMutHotspot = geneMaf[geneMaf['is-a-hotspot'] == 'Y'].shape[0]
    qVal = None
    if gene in qValDict:
        qVal = qValDict[gene]
    listOfDicts.append({'gene': gene, 'qVal': qVal, 'nHotspotMuts': nMutHotspot, 'nOncogenicMuts': nMutOnc})

df = pd.DataFrame(listOfDicts)

In [34]:
df['displayLabelHotspot'] = df.apply(lambda row: row['gene'] if (row['nHotspotMuts'] > 5) else None, axis=1)
df['displayLabelOncogenic'] = df.apply(lambda row: row['gene'] if (row['nOncogenicMuts'] > 20) else None, axis=1)

In [35]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/dndscv_pancan_vs_local_genes.tsv', index=False, sep='\t')

**Format DNDSCV data for plotting**

In [17]:
dfEndo = pd.read_table('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/endometrialDNDSdata.tsv')
dfColo = pd.read_table('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/colorectalDNDSdata.tsv')
dfGlioma = pd.read_table('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myAdjustedDataFiles/gliomaDNDSdata.tsv')


In [18]:
dfEndo['cancerType'] = 'Endometrial Cancer'
dfColo['cancerType'] = 'Colorectal Cancer'
dfGlioma['cancerType'] = 'Glioma'

In [19]:
df = pd.concat([dfEndo, dfColo, dfGlioma])

In [20]:
df['displayLabel'] = df.apply(lambda row: row['gene_name']
                 if (row['qglobal_cv.Hypermutated'] < .05 or row['qglobal_cv.Normal'] < 0) else None, axis=1)

In [21]:
df.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/dndscvSummary.tsv', index=False, sep='\t')

**Comparing hypermutation gene fractions with hypermutated dndscv results**

In [26]:
#dfObserved = pd.read_table('/Users/friedman/Desktop/WORK/dataForLocalPlotting/mutIncidenceComparisson.tsv')
dfObserved.columns.values

array(['cancerType', 'gene', 'geneSize', 'nmutPerCaseHyper',
       'nmutPerCaseNormal', 'displayLabel', 'enrichmentStatus'],
      dtype=object)

In [27]:
listOfDicts = []
for gene in set(df['gene_name']):
    if gene in set(dfObserved['gene']):
        for cancerType in ['Endometrial Cancer', 'Colorectal Cancer', 'Glioma']:
            ctGeneDf = dfObserved[(dfObserved['gene'] == gene) & (dfObserved['cancerType'] == cancerType)]
            fracCaseObs = ctGeneDf['nmutPerCaseHyper'].iloc[0]
            geneSize = ctGeneDf['geneSize'].iloc[0]
            
            dndsCtGeneDf = df[(df['gene_name'] ==  gene) & (df['cancerType'] == cancerType)]
            qVal = dndsCtGeneDf['qglobal_cv.Hypermutated'].iloc[0] 
            
            listOfDicts.append({'cancerType': cancerType, 'gene': gene, 'fracCase': fracCaseObs, 
                                'geneSize': geneSize, 'qVal': qVal})
            
dfComp = pd.DataFrame(listOfDicts)
        

In [54]:
dfComp['displayLabel'] = dfComp.apply(lambda row: row['gene'] if ((row['qVal'] > .05 and row['fracCase'] > .2) or (row['qVal'] < .1)) else None, axis=1)

In [55]:
dfCompToWrite = dfComp[(dfComp['cancerType'] != 'Glioma') & (dfComp['qVal'] > .05)]

In [56]:
dfCompToWrite.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/dndsVsFracMutComp.tsv', index=False, sep='\t')